In [ ]:
gwutils.optimal_snr_squared(
            signal=signal[self.strain_data.frequency_mask],
            power_spectral_density=self.power_spectral_density_array[self.strain_data.frequency_mask],
            duration=self.strain_data.duration)

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import bilby

In [7]:
injection_parameters = dict(
    mass_1=36., mass_2=29., a_1=0.4, a_2=0.3, tilt_1=0.5, tilt_2=1.0,
    phi_12=1.7, phi_jl=0.3, luminosity_distance=4000., theta_jn=0.4, psi=2.659,
    phase=1.3, geocent_time=1126259642.413, ra=1.375, dec=-1.2108)

In [8]:
waveform_arguments = dict(waveform_approximant='IMRPhenomPv2',
                          reference_frequency=50., minimum_frequency=20., catch_waveform_errors=True)
duration = 4.
sampling_frequency = 2048.

In [9]:
waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration, sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=waveform_arguments)

03:49 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


In [5]:
ifos = bilby.gw.detector.InterferometerList(['H1', 'L1'])
ifos.set_strain_data_from_power_spectral_densities(
    sampling_frequency=sampling_frequency, duration=duration,
    start_time=injection_parameters['geocent_time'] - 3)
injection = ifos.inject_signal(
    waveform_generator=waveform_generator,
    parameters=injection_parameters)

06:17 bilby INFO    : Injected signal in H1:
06:17 bilby INFO    :   optimal SNR = 5.89
06:17 bilby INFO    :   matched filter SNR = 7.61+0.10j
06:17 bilby INFO    :   mass_1 = 36.0
06:17 bilby INFO    :   mass_2 = 29.0
06:17 bilby INFO    :   a_1 = 0.4
06:17 bilby INFO    :   a_2 = 0.3
06:17 bilby INFO    :   tilt_1 = 0.5
06:17 bilby INFO    :   tilt_2 = 1.0
06:17 bilby INFO    :   phi_12 = 1.7
06:17 bilby INFO    :   phi_jl = 0.3
06:17 bilby INFO    :   luminosity_distance = 4000.0
06:17 bilby INFO    :   theta_jn = 0.4
06:17 bilby INFO    :   psi = 2.659
06:17 bilby INFO    :   phase = 1.3
06:17 bilby INFO    :   geocent_time = 1126259642.413
06:17 bilby INFO    :   ra = 1.375
06:17 bilby INFO    :   dec = -1.2108
06:17 bilby INFO    : Injected signal in L1:
06:17 bilby INFO    :   optimal SNR = 4.76
06:17 bilby INFO    :   matched filter SNR = 4.67-0.95j
06:17 bilby INFO    :   mass_1 = 36.0
06:17 bilby INFO    :   mass_2 = 29.0
06:17 bilby INFO    :   a_1 = 0.4
06:17 bilby INFO   

In [11]:
ifos[0].meta_data["optimal_SNR"]

5.888842359288943

In [ ]:
https://github.com/lscsoft/bilby/blob/b1e02f1dfae03d4939cae9c95eff300c22919689/bilby/gw/detector/interferometer.py#L247
https://github.com/lscsoft/bilby/blob/b1e02f1dfae03d4939cae9c95eff300c22919689/bilby/gw/utils.py#L85

In [19]:
from bilby.gw import utils as gwutils

In [27]:
from math import fmod
from bilby.core.utils import ra_dec_to_theta_phi

def greenwich_mean_sidereal_time(time):
    from lal import GreenwichMeanSiderealTime
    time = float(time)
    return GreenwichMeanSiderealTime(time)

In [25]:
np.ones(2)*1246527184.169434

array([1.24652718e+09, 1.24652718e+09])

In [26]:
# https://git.ligo.org/lscsoft/lalsuite/-/blob/master/lal/test/tools/IndependentDetResponseTest.c#L958
greenwich_mean_sidereal_time(1246527184.169434)

44907.11676977647

In [ ]:
10e-3*100000/3600

Signature: fmod(x, y, /)
Docstring:
Return fmod(x, y), according to platform C.

x % y may differ.
Type:      builtin_function_or_method

In [21]:
def ra_dec_to_theta_phi(ra, dec, gmst):
    phi = ra - gmst
    theta = np.pi / 2 - dec
    return theta, phi

gmst = fmod(greenwich_mean_sidereal_time(time), 2 * np.pi)
theta, phi = ra_dec_to_theta_phi(ra, dec, gmst)
u = np.array([np.cos(phi) * np.cos(theta), np.cos(theta) * np.sin(phi), -np.sin(theta)])
v = np.array([-np.sin(phi), np.cos(phi), 0])
m = -u * np.sin(psi) - v * np.cos(psi)
n = -u * np.cos(psi) + v * np.sin(psi)

Signature: np.einsum(*operands, out=None, optimize=False, **kwargs)
Docstring:
einsum(subscripts, *operands, out=None, dtype=None, order='K',
       casting='safe', optimize=False)

Evaluates the Einstein summation convention on the operands.

Using the Einstein summation convention, many common multi-dimensional,
linear algebraic array operations can be represented in a simple fashion.
In *implicit* mode `einsum` computes these values.

In *explicit* mode, `einsum` provides further flexibility to compute
other array operations that might not be considered classical Einstein
summation operations, by disabling, or forcing summation over specified
subscript labels.

See the notes and examples for clarification.

Parameters
----------
subscripts : str
    Specifies the subscripts for summation as comma separated list of
    subscript labels. An implicit (classical Einstein summation)
    calculation is performed unless the explicit indicator '->' is
    included as well as subscript label

In [42]:
import datetime

def GPS_to_JulianDay(gps_time):
    utc_time = datetime.datetime.fromtimestamp(gps_time)
    julian_day = 2440587.5 + (utc_time - datetime.datetime(2000, 1, 1)).total_seconds() / 86400.0
    return julian_day

def GreenwichMeanSiderealTime(gps_time, equation_of_equinoxes):
    # Convert GPS time to Julian Day
    julian_day = GPS_to_JulianDay(gps_time)

    # Calculate centuries since J2000 epoch
    t = (julian_day - 2451545.0) / 36525.0

    # Calculate GMST in seconds
    gmst = 67310.54841 + (876600.0 * 3600.0 + 8640184.812866) * t + 0.093104 * t**2 - 6.2e-6 * t**3

    # Add equation of equinoxes in seconds
    gmst += equation_of_equinoxes

    # Convert GMST to radians
    gmst_radians = gmst * (2 * 3.14159265358979323846) / 86400.0

    return gmst_radians

# Example usage:
gps_time = 1246527184.169434  # Replace with your GPS time in seconds
equation_of_equinoxes = 0  # Replace with your equation of equinoxes in seconds
gmst_radians = GreenwichMeanSiderealTime(gps_time, equation_of_equinoxes)
print("Greenwich Mean Sidereal Time (radians):", gmst_radians)




Greenwich Mean Sidereal Time (radians): -47164.653466816046


In [ ]:
365*24*60*60*50

In [95]:
import bilby
import numpy as np
from numba import njit, jit

C = 299792458.0
G = 6.67408 * 1e-11
Mo = 1.989 * 1e30
Gamma = 0.5772156649015329
Pi = np.pi
MTSUN_SI = 4.925491025543576e-06

In [23]:
@njit
def einsum1(m,n):
    ans = np.zeros((3,3))
    ans[0,0] = m[0]*n[0]
    ans[0,1] = m[0]*n[1]
    ans[0,2] = m[0]*n[2]
    ans[1,0] = m[1]*n[0]
    ans[1,1] = m[1]*n[1]
    ans[1,2] = m[1]*n[2]
    ans[2,0] = m[2]*n[0]
    ans[2,1] = m[2]*n[1]
    ans[2,2] = m[2]*n[2]
    return ans
@njit
def einsum2(m,n):
    ans = m[0,0]*n[0,0] + m[0,1]*n[0,1] + m[0,2]*n[0,2] + m[1,0]*n[1,0] + m[1,1]*n[1,1] + m[1,2]*n[1,2] + m[2,0]*n[2,0] + m[2,1]*n[2,1] + m[2,2]*n[2,2]
    return ans

@njit
def gps_to_gmst(gps_time):
    slope = 7.292115855382993e-05
    time0 = 1126259642.413
    time = gps_time - time0
    return slope*time+36137.068361399164

@njit
def ra_dec_to_theta_phi(ra, dec, gmst):
    phi = ra - gmst
    theta = np.pi / 2.0 - dec
    return theta, phi

@njit
def get_polarization_tensor(ra, dec, time, psi, mode='plus'):
    gmst = np.fmod(gps_to_gmst(time), 2 * np.pi)
    theta, phi = ra_dec_to_theta_phi(ra, dec, gmst)
    u = np.array([np.cos(phi) * np.cos(theta), np.cos(theta) * np.sin(phi), -np.sin(theta)])
    v = np.array([-np.sin(phi), np.cos(phi), 0])
    m = -u * np.sin(psi) - v * np.cos(psi)
    n = -u * np.cos(psi) + v * np.sin(psi)

    if mode == 'plus':
        return einsum1(m, m) - einsum1(n, n)
    elif mode == 'cross':
        return einsum1(m, n) + einsum1(n, m)

@njit
def antenna_response(ra, dec, time, psi, detector_tensor, mode='plus'):

    polarization_tensor = get_polarization_tensor(ra, dec, time, psi, mode=mode)
    return einsum2(detector_tensor, polarization_tensor)

@njit
def antenna_response_array(ra, dec, time, psi, detector_tensor, mode='plus'):

    polarization_tensor = get_polarization_tensor(ra, dec, time, psi, mode=mode)
    ans=einsum2(detector_tensor, polarization_tensor)

In [24]:
# define values
time = 1126259642.413
ra = 1.375
dec = -1.2108
psi = 0.1
mode = "plus"
ifos = bilby.gw.detector.InterferometerList(['H1', 'L1'])
detector_tensor = ifos[0].detector_tensor

In [26]:
antenna_response(ra, dec, time, psi, detector_tensor, mode='cross')

0.5911086754190837

In [34]:
waveform_arguments

{'waveform_approximant': 'IMRPhenomPv2',
 'reference_frequency': 50.0,
 'minimum_frequency': 20.0,
 'catch_waveform_errors': True}

In [1]:
from bilby.gw.utils import (lalsim_GetApproximantFromString,
                    lalsim_SimInspiralFD,
                    lalsim_SimInspiralChooseFDWaveform,
                    lalsim_SimInspiralWaveformParamsInsertTidalLambda1,
                    lalsim_SimInspiralWaveformParamsInsertTidalLambda2,
                    lalsim_SimInspiralChooseFDWaveformSequence)
import lal
import lalsimulation as lalsim
from bilby.gw.conversion import bilby_to_lalsimulation_spins

parsec = 3.085677581491367e+16
solar_mass = 1.9884098706980507e+30

In [10]:
waveform_generator.frequency_array

array([0.00000e+00, 2.50000e-01, 5.00000e-01, ..., 1.02350e+03,
       1.02375e+03, 1.02400e+03])

In [30]:
# inlcude maximum frequency
num_samples = int((1024 - 0) / (1/4)) + 1
arr_ = np.linspace(0, 1024, num_samples)

In [24]:
# first non-zero value in an array 
def first_nonzero(arr, axis, invalid_val=-1):
    mask = arr!=0
    return np.where(mask.any(axis=axis), mask.argmax(axis=axis), invalid_val)

first_nonzero(waveform_generator.frequency_array, axis=0)

array(1)

In [31]:
np.sum(waveform_generator.frequency_array-arr_)

0.0

In [39]:
approximant='IMRPhenomXPHM'
duration = 4.
df = 1.0 / duration
sampling_frequency = 2048.
f_min=20.0
f_max=sampling_frequency/2
num_samples = int(f_max / df) + 1
frequency_array = np.linspace(0, f_max, num_samples)
mass_1 = 36.0
mass_2 = 29.0
luminosity_distance = 200.0
theta_jn = 0.4
phase = 1.2
a_1=0.4
a_2=0.3
tilt_1=0.5
tilt_2=1.0
phi_12=1.7
phi_jl=0.3
# a_1=0.0
# a_2=0.0
# tilt_1=0.0
# tilt_2=0.0
# phi_12=0.0
# phi_jl=0.0
lambda_1=0.0
lambda_2=0.0
eccentricity=0.0

waveform_kwargs = dict(
        waveform_approximant=approximant, reference_frequency=20.0,
        minimum_frequency=f_min, maximum_frequency=frequency_array[-1],
        catch_waveform_errors=False, pn_spin_order=-1, pn_tidal_order=-1,
        pn_phase_order=-1, pn_amplitude_order=0)

waveform_approximant = waveform_kwargs['waveform_approximant']
reference_frequency = waveform_kwargs['reference_frequency']
minimum_frequency = waveform_kwargs['minimum_frequency']
maximum_frequency = waveform_kwargs['maximum_frequency']
catch_waveform_errors = waveform_kwargs['catch_waveform_errors']
pn_spin_order = waveform_kwargs['pn_spin_order']
pn_tidal_order = waveform_kwargs['pn_tidal_order']
pn_phase_order = waveform_kwargs['pn_phase_order']
pn_amplitude_order = waveform_kwargs['pn_amplitude_order']
waveform_dictionary = waveform_kwargs.get(
    'lal_waveform_dictionary', lal.CreateDict()
)
approximant = lalsim_GetApproximantFromString(waveform_approximant)
start_frequency = minimum_frequency
delta_frequency = frequency_array[1] - frequency_array[0]

frequency_bounds = ((frequency_array >= minimum_frequency) *
                        (frequency_array <= maximum_frequency))
luminosity_distance = luminosity_distance * 1e6 * parsec
mass_1 = mass_1 * solar_mass
mass_2 = mass_2 * solar_mass

iota, spin_1x, spin_1y, spin_1z, spin_2x, spin_2y, spin_2z = bilby_to_lalsimulation_spins(
        theta_jn=theta_jn, phi_jl=phi_jl, tilt_1=tilt_1, tilt_2=tilt_2,
        phi_12=phi_12, a_1=a_1, a_2=a_2, mass_1=mass_1, mass_2=mass_2,
        reference_frequency=reference_frequency, phase=phase)

longitude_ascending_nodes = 0.0
mean_per_ano = 0.0

lalsim.SimInspiralWaveformParamsInsertPNSpinOrder(
    waveform_dictionary, int(pn_spin_order))
lalsim.SimInspiralWaveformParamsInsertPNTidalOrder(
    waveform_dictionary, int(pn_tidal_order))
lalsim.SimInspiralWaveformParamsInsertPNPhaseOrder(
    waveform_dictionary, int(pn_phase_order))
lalsim.SimInspiralWaveformParamsInsertPNAmplitudeOrder(
    waveform_dictionary, int(pn_amplitude_order))
lalsim_SimInspiralWaveformParamsInsertTidalLambda1(
    waveform_dictionary, lambda_1)
lalsim_SimInspiralWaveformParamsInsertTidalLambda2(
    waveform_dictionary, lambda_2)

if lalsim.SimInspiralImplementedFDApproximants(approximant):
    wf_func = lalsim_SimInspiralChooseFDWaveform
else:
    wf_func = lalsim_SimInspiralFD

hplus, hcross = wf_func(
    mass_1, mass_2, spin_1x, spin_1y, spin_1z, spin_2x, spin_2y,
    spin_2z, luminosity_distance, iota, phase,
    longitude_ascending_nodes, eccentricity, mean_per_ano, delta_frequency,
    start_frequency, maximum_frequency, reference_frequency,
    waveform_dictionary, approximant)


In [40]:
spin_1x

0.017225649380610542

In [38]:
spin_1x

0.04510383632050702

In [35]:
len(hplus.data.data)

4097

In [36]:
size = 1000
mass_1_ = 36.0*np.ones(size)* solar_mass
mass_2_ = 29.0*np.ones(size)* solar_mass
luminosity_distance_ = 200.0*np.ones(size)* 1e6 * parsec
theta_jn_ = 0.4*np.ones(size)
phase_ = 1.2*np.ones(size)
a_1_=0.4*np.ones(size)
a_2_=0.3*np.ones(size)
tilt_1_=0.5*np.ones(size)
tilt_2_=1.0*np.ones(size)
phi_12_=1.7*np.ones(size)
phi_jl_=0.3*np.ones(size)
lambda_1_=0.0*np.ones(size)
lambda_2_=0.0*np.ones(size)
eccentricity_=0.0*np.ones(size)

hplus_arr, hcross_arr = [],[]
zero_complex_arr = np.zeros_like(frequency_array, dtype=complex)

for i in range(size):
        mass_1 = mass_1_[i]
        mass_2 = mass_2_[i]
        luminosity_distance = luminosity_distance_[i]
        theta_jn = theta_jn_[i]
        phase = phase_[i]
        a_1=a_1_[i]
        a_2=a_2_[i]
        tilt_1=tilt_1_[i]
        tilt_2=tilt_2_[i]
        phi_12=phi_12_[i]
        phi_jl=phi_jl_[i]
        lambda_1=lambda_1_[i]
        lambda_2=lambda_2_[i]
        eccentricity=eccentricity_[i]

        iota, spin_1x, spin_1y, spin_1z, spin_2x, spin_2y, spin_2z =\
        bilby_to_lalsimulation_spins(
        theta_jn=theta_jn, phi_jl=phi_jl, tilt_1=tilt_1, tilt_2=tilt_2,
        phi_12=phi_12, a_1=a_1, a_2=a_2, mass_1=mass_1, mass_2=mass_2,
        reference_frequency=reference_frequency, phase=phase)

        hplus, hcross = wf_func(mass_1, mass_2, spin_1x, spin_1y, spin_1z, spin_2x, spin_2y,spin_2z, luminosity_distance, iota, phase,longitude_ascending_nodes, eccentricity, mean_per_ano, delta_frequency,start_frequency, maximum_frequency, reference_frequency,waveform_dictionary, approximant)

        h_plus = zero_complex_arr
        h_cross = zero_complex_arr
        h_plus[:len(hplus.data.data)] = hplus.data.data
        h_cross[:len(hcross.data.data)] = hcross.data.data
        h_plus *= frequency_bounds
        h_cross *= frequency_bounds
        hplus_arr.append(h_plus)
        hcross_arr.append(h_cross)

# now do it with multiprocessing

* 53.4s for 10,000 samples

In [ ]:

        hplus, hcross = wf_func(mass_1, mass_2, spin_1x, spin_1y, spin_1z, spin_2x, spin_2y,spin_2z, luminosity_distance, iota, phase,longitude_ascending_nodes, eccentricity, mean_per_ano, delta_frequency,start_frequency, maximum_frequency, reference_frequency,waveform_dictionary, approximant)
        h_plus = zero_complex_arr
        h_cross = zero_complex_arr
        h_plus[:len(hplus.data.data)] = hplus.data.data
        h_cross[:len(hcross.data.data)] = hcross.data.data
        h_plus *= frequency_bounds
        h_cross *= frequency_bounds
        hplus_arr.append(h_plus)
        hcross_arr.append(h_cross)

In [86]:
h_plus = np.zeros_like(frequency_array, dtype=complex)
h_cross = np.zeros_like(frequency_array, dtype=complex)
h_plus[:len(hplus.data.data)] = hplus.data.data
h_cross[:len(hcross.data.data)] = hcross.data.data

In [87]:
def transform_precessing_spins(theta_jn, phi_jl, tilt_1, tilt_2, phi_12, a_1,
                               a_2, mass_1, mass_2, reference_frequency, phase):

    iota, spin_1x, spin_1y, spin_1z, spin_2x, spin_2y, spin_2z = (
        lalsim_SimInspiralTransformPrecessingNewInitialConditions(
            theta_jn, phi_jl, tilt_1, tilt_2, phi_12, a_1, a_2, mass_1, mass_2,
            reference_frequency, phase))

    return iota, spin_1x, spin_1y, spin_1z, spin_2x, spin_2y, spin_2z

array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j])